# Типы сервисов

- Пример предназначен для демонстрации работы по получению списка типов сервисов для выбранного `scenario_id`. 
- Особенность заключается в маппинге или адаптации типов сервисов из БД к типам сервисов `blocksnet`.

Список сервисов **уникален** для каждого `project_id`.

In [47]:
SCENARIO_ID = 1392

In [48]:
from prostor.fetchers import scenarios

PROJECT_ID = scenarios.get_scenario(SCENARIO_ID)['project']['project_id']
PROJECT_ID

73

## 1. Получение списка сервисов

### 1.2. Получение списка нормативов

Надо как-то получить `normatives` как с эндпоинтом [/api/v1/territory/{territory_id}/normatives](http://10.32.1.65:5300/api/docs#/territories/get_territory_normatives_api_v1_territory__territory_id__normatives_get).

В данном примере цепляю по сути для `territory` из [/api/v1/projects/{project_id}](http://10.32.1.65:5300/api/docs#/projects/get_project_by_id_api_v1_projects__project_id__get). Может стоит спускаться пониже.

In [49]:
from prostor.fetchers import projects

TERRITORY_ID = projects.get_project(PROJECT_ID)['territory']['id']
TERRITORY_ID

1

In [50]:
from prostor.fetchers import territories

normatives_df = territories.get_territory_normatives(TERRITORY_ID)[[
    'radius_availability_meters',
    'time_availability_minutes',
    'services_per_1000_normative',
    'services_capacity_per_1000_normative'
]].copy()
normatives_df.head()

,radius_availability_meters,time_availability_minutes,services_per_1000_normative,services_capacity_per_1000_normative
service_type_id,,,,
128,None,60,NaN,257.0
104,None,15,NaN,10.0
115,None,40,NaN,96.0
86,None,60,NaN,150.0
62,None,30,NaN,71.0


### 1.2. Получение списка сервисов

In [51]:
import prostor.fetchers.misc as misc

service_types_df = misc.get_service_types()
service_types_df['weight'] = service_types_df['properties'].apply(lambda p : p['weight_value'] if 'weight_value' in p else None)
service_types_df = service_types_df[[
    'capacity_modeled',
    'infrastructure_type',
    'weight',
]].copy()
service_types_df.head()

,capacity_modeled,infrastructure_type,weight
service_type_id,,,
1,2000.0,basic,0.2
2,NaN,None,NaN
3,NaN,None,NaN
4,NaN,None,NaN
5,NaN,comfort,0.5


### 1.3. Дружба нормативов и списка сервисов.

Всё собираем в один большой список, чтобы дальше с ним работать.

In [52]:
service_types_df = service_types_df.join(normatives_df)

In [53]:
service_types_df.head()

,capacity_modeled,infrastructure_type,weight,radius_availability_meters,time_availability_minutes,services_per_1000_normative,services_capacity_per_1000_normative
service_type_id,,,,,,,
1,2000.0,basic,0.2,None,30.0,NaN,150.0
2,NaN,None,NaN,NaN,NaN,NaN,NaN
3,NaN,None,NaN,NaN,NaN,NaN,NaN
4,NaN,None,NaN,NaN,NaN,NaN,NaN
5,NaN,comfort,0.5,None,60.0,NaN,64.0


### 1.4. Получение списка ценностей

По возможности цепляем каждому service_type_id его список ценностей (`soc_value_id`)

In [54]:
def _get_social_values_ids(service_type_id : int) -> list[int]:
    try:
        social_values_df = misc.get_service_type_social_values(service_type_id)
        return list(social_values_df.index)
    except:
        return None
    
service_types_df['soc_values_ids'] = service_types_df.apply(lambda s : _get_social_values_ids(s.name), axis=1)

In [55]:
service_types_df.head()

,capacity_modeled,infrastructure_type,weight,radius_availability_meters,time_availability_minutes,services_per_1000_normative,services_capacity_per_1000_normative,soc_values_ids
service_type_id,,,,,,,,
1,2000.0,basic,0.2,None,30.0,NaN,150.0,[8]
2,NaN,None,NaN,NaN,NaN,NaN,NaN,None
3,NaN,None,NaN,NaN,NaN,NaN,NaN,None
4,NaN,None,NaN,NaN,NaN,NaN,NaN,None
5,NaN,comfort,0.5,None,60.0,NaN,64.0,None


### 1.5. Адаптация к блокснет

По возможности даем списку сервисов нейминги из блокснета, чтобы смочь сдружить с оптимизатором и прочим. 

Обычная обеспеченность (`competitive_provision`) отработает и без этого, но ей в Ф36 нужен полный список всё же.

ВНИМАНИЕ! В `v1.0.0.a8` появился `college`.

In [56]:
SERVICE_TYPES_MAPPING = {
    # basic
    1: 'park',
    21: 'kindergarten',
    22: 'school',
    28: 'polyclinic',
    34: 'pharmacy',
    61: 'cafe',
    66: 'pitch',
    68: None, # спортивный зал
    74: 'playground',
    78: 'police',
    # additional
    30 : None, # стоматология
    35 : 'hospital',
    50 : 'museum',
    56 : 'cinema',
    57 : 'mall', 
    59 : 'stadium',
    62 : 'restaurant',
    63 : 'bar',
    77 : None, # скейт парк
    79 : None, # пожарная станция
    80 : 'train_station',
    89 : 'supermarket',
    99 : None, # пункт выдачи
    100 : 'bank',
    107 : 'veterinary',
    143 : 'sanatorium',
    # comfort
    5 : 'beach',
    27 : 'university',
    36 : None, # роддом
    48 : 'library',
    51 : 'theatre',
    91 : 'market',
    93 : None, # одежда и обувь
    94 : None, # бытовая техника
    95 : None, # книжный магазин
    96 : None, # детские товары
    97 : None, # спортивный магазин
    108 : None, # зоомагазин
    110 : 'hotel',
    114 : 'religion', # религиозный объект
    # others
    26 : 'college', # ССУЗ
    32 : None, # женская консультация
    39 : None, # скорая помощь
    40 : None, # травматология
    45 : 'recruitment',
    47 : 'multifunctional_center',
    55 : 'zoo',
    65 : 'bakery',
    67 : 'swimming_pool',
    75 : None, # парк аттракционов
    81 : 'train_building',
    82 : 'aeroway_terminal', # аэропорт??
    86 : 'bus_station',
    88 : 'subway_entrance',
    102 : 'lawyer',
    103 : 'notary',
    109 : 'dog_park',
    111 : 'hostel',
    112 : None, # база отдыха
    113 : None, # памятник
}

In [57]:
service_types_df['blocksnet'] = service_types_df.apply(lambda s : SERVICE_TYPES_MAPPING.get(s.name), axis=1)
service_types_df.head()

,capacity_modeled,infrastructure_type,weight,radius_availability_meters,time_availability_minutes,services_per_1000_normative,services_capacity_per_1000_normative,soc_values_ids,blocksnet
service_type_id,,,,,,,,,
1,2000.0,basic,0.2,None,30.0,NaN,150.0,[8],park
2,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
3,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
4,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
5,NaN,comfort,0.5,None,60.0,NaN,64.0,None,beach


## 2. Сохранение списка

In [58]:
service_types_df

,capacity_modeled,infrastructure_type,weight,radius_availability_meters,time_availability_minutes,services_per_1000_normative,services_capacity_per_1000_normative,soc_values_ids,blocksnet
service_type_id,,,,,,,,,
1,2000.0,basic,0.2,None,30.0,NaN,150.0,[8],park
2,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
3,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
4,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
5,NaN,comfort,0.5,None,60.0,NaN,64.0,None,beach
...,...,...,...,...,...,...,...,...,...
134,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
135,NaN,None,NaN,NaN,NaN,NaN,NaN,None,None
136,NaN,None,NaN,None,15.0,NaN,53.0,None,None


In [59]:
service_types_df.to_pickle('./data/service_types.pickle')